In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False


In [4]:
#**1.加载前列腺癌数据，构建会话对象,lpsa读取样本数据，并且将lable连续型变量转化为0,1 [0代表正常，1代表异常](5分)
from sklearn.preprocessing import Binarizer
data = pd.read_csv("lpsa.data")
binarizer = Binarizer(threshold=0)
data["LabeledPoint"] = binarizer.fit_transform(data.iloc[:,0].values.reshape(-1,1))
data

,-0.4307829,-1.63735562648104,-2.00621178480549,-1.86242597251066,-1.02470580167082,-0.522940888712441,-0.863171185425945,-1.04215728919298,-0.864466507337306,LabeledPoint
0,-0.162519,-1.988980,-0.722009,-0.787896,-1.024706,-0.522941,-0.863171,-1.042157,-0.864467,0.0
1,-0.162519,-1.578819,-2.188784,1.361163,-1.024706,-0.522941,-0.863171,0.342627,-0.155348,0.0
2,-0.162519,-2.166917,-0.807994,-0.787896,-1.024706,-0.522941,-0.863171,-1.042157,-0.864467,0.0
3,0.371564,-0.507874,-0.458834,-0.250631,-1.024706,-0.522941,-0.863171,-1.042157,-0.864467,1.0
4,0.765468,-2.036128,-0.933955,-1.862426,-1.024706,-0.522941,-0.863171,-1.042157,-0.864467,1.0
...,...,...,...,...,...,...,...,...,...,...
61,4.029806,1.609063,1.103786,0.555266,-1.024706,-0.522941,-0.863171,-1.042157,-0.864467,1.0
62,4.129551,1.003621,0.113497,-0.384948,0.860016,1.892548,-0.863171,0.342627,-0.332628,1.0
63,4.385147,1.255920,0.577607,0.555266,-1.024706,1.892548,1.073572,0.342627,1.262889,1.0
64,4.684443,2.096506,0.625489,-2.668323,-1.024706,1.892548,1.679542,0.342627,0.553770,1.0


In [5]:
#**2训练时需要LabeledPoint这样格式的数据，自主划分训练集、测试集(5分)
from sklearn.model_selection import train_test_split
X = data.iloc[:,1:9]
y = data["LabeledPoint"]
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
#**3.自主选择两种算法，构造训练模型(8分)
# **使用随机森林 跟 决策树 
#**4、对以上两种算法进行参数调优（8分）
#**5、对以上两种模型评估泛化能力，至少使用两种评估指标（8分）
#**6、输出以上两种模型最优参数（5分）
#**7、使用评估指标较优者，对样本批量预测（5分）
#**8  使用classification_report输出分类报告结果（4分）

In [6]:
#**1  导入相关的库（4分）
from surprise import Dataset,Reader
from surprise import KNNBasic

In [21]:
#**2  自行定义基于物品或者用户推荐的函数（4分）
def user_test(user_id,trainset,testset,k=10):
    ml = KNNBasic(sim_options={"user_based":True})
    ml.fit(trainset)
    user_predicts = ml.test(testset)
    user = [pred for pred in user_predicts if pred.uid == user_id]
    user.sort(key=lambda x:x.est,reverse=True)
    return [pred.iid for pred in user[:k]],user_predicts

def item_test(item_id,trainset,testset,k=10):
    ml = KNNBasic(sim_options={"user_based":True})
    ml.fit(trainset)
    user_predicts = ml.test(testset)
    user = [pred for pred in user_predicts if pred.iid == item_id]
    user.sort(key=lambda x:x.est,reverse=True)
    return [pred.uid for pred in user[:k]],user_predicts

In [19]:
#**3  读取数据 其中用户 电影评分及评分时间为附件所示（4分）
data = pd.read_csv("sample_movielens_ratings.txt",sep="::",names=["uid","iid","est","time"])
read = Reader(rating_scale=(1,5))
df = Dataset.load_from_df(data.drop(columns="time"),read)

C:\Users\lijunhao\AppData\Local\Temp\ipykernel_10988\3257941382.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv("sample_movielens_ratings.txt",sep="::",names=["uid","iid","est","time"])


In [14]:
#**4  生成用户-物品评分矩阵（4分）
print(data[["uid", "iid", "est"]])

      uid  iid  est
0       0    2    3
1       0    3    1
2       0    5    2
3       0    9    4
4       0   11    1
...   ...  ...  ...
1496   29   90    4
1497   29   93    1
1498   29   94    4
1499   29   97    1
1500   29   99    1

[1501 rows x 3 columns]


In [22]:
#**5  计算相似度：  
#**从基于物品相似度计算（4分）
#**   从基于用户相似度计算（4分）
from surprise.model_selection import train_test_split
trainset,testset = train_test_split(df,test_size=0.2,random_state=42)
user_top10,user_pred = user_test(0,trainset,testset)
item_top10,item_pred = item_test(0,trainset,testset)
user = pd.DataFrame(user_pred)
item = pd.DataFrame(item_pred)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


In [23]:
#**6  进行相似度预测:调用第2步进行相似度计算
#**从基于物品相似度计算（4分）
#**    从基于用户相似度计算（4分）
user[user['uid'] == 0]

,uid,iid,r_ui,est,details
11,0,51,1.0,1.877850,"{'actual_k': 17, 'was_impossible': False}"
15,0,77,2.0,2.392023,"{'actual_k': 9, 'was_impossible': False}"
53,0,19,1.0,1.619435,"{'actual_k': 11, 'was_impossible': False}"
109,0,3,1.0,1.319781,"{'actual_k': 10, 'was_impossible': False}"
185,0,87,1.0,2.101785,"{'actual_k': 11, 'was_impossible': False}"
200,0,29,1.0,2.312419,"{'actual_k': 15, 'was_impossible': False}"
234,0,69,1.0,1.939611,"{'actual_k': 12, 'was_impossible': False}"
246,0,28,1.0,2.053947,"{'actual_k': 9, 'was_impossible': False}"
253,0,44,1.0,1.402291,"{'actual_k': 13, 'was_impossible': False}"
271,0,45,2.0,1.222323,"{'actual_k': 15, 'was_impossible': False}"


In [24]:
item[item["iid"] == 0]

,uid,iid,r_ui,est,details
39,10,0,3.0,1.343411,"{'actual_k': 11, 'was_impossible': False}"
171,19,0,1.0,1.094313,"{'actual_k': 11, 'was_impossible': False}"
179,21,0,1.0,1.172975,"{'actual_k': 11, 'was_impossible': False}"
232,22,0,1.0,1.122501,"{'actual_k': 11, 'was_impossible': False}"
282,15,0,1.0,1.155047,"{'actual_k': 11, 'was_impossible': False}"


In [25]:
#**7  显示给用户推荐的top10的电影（5分）
print(f"用户推荐top10{user_top10}")
print(f"商品推荐top10{item_top10}")

用户推荐top10[77, 29, 87, 28, 69, 51, 19, 44, 3, 45]
商品推荐top10[10, 21, 15, 22, 19]


In [27]:
#**8   进行性能评估并输出结果（5分）
from surprise import accuracy
rmse = accuracy.rmse(user_pred)
rmse

RMSE: 1.1174


1.1173735432024514

In [28]:
#**9  手动实现简单的逻辑回归算法（10分）
def sigmoid(x):
    return 1/(1+np.exp(-x))

def log_cost(x,y,theta):
    m = y.size 
    h = sigmoid(x.dot(theta))
    j = (-1/m)*(y.dot(np.log(h)) + (1-y).dot(np.log(1-h)))
    return j
def log_desc(x,y,theta,lr,num_iter):
    m = y.size
    arr = np.zeros(num_iter)
    for i in range(num_iter):
        h = sigmoid(x.dot(theta))
        arr[i] =log_cost(x,y,theta)
        grid = (1/m)*(x.T.dot(h-y))
        theta -= lr*grid
    return theta,arr


In [30]:
from sklearn.datasets import  make_classification
from sklearn.model_selection import train_test_split
X,y = make_classification(n_samples=300,n_features=5,n_classes=2)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
theta = np.zeros(X_train.shape[1])
theta,arr = log_desc(X_train,y_train,theta,0.01,10000)
print(arr)
pred = sigmoid(X_test.dot(theta)).round()
pred

[0.69314718 0.69067741 0.68823106 ... 0.34040841 0.34040841 0.34040841]


array([0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
       1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1.,
       0., 0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 1.,
       1., 0., 1., 0., 1., 0., 1., 1., 0.])